##### Copyright 2023 The TF-Agents Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Redes

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/agents/tutorials/8_networks_tutorial">     <img src="https://www.tensorflow.org/images/tf_logo_32px.png">     Ver em TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/agents/tutorials/8_networks_tutorial.ipynb">     <img src="https://www.tensorflow.org/images/colab_logo_32px.png">     Executar no Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/agents/tutorials/8_networks_tutorial.ipynb">     <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">     Ver fonte no GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/agents/tutorials/8_networks_tutorial.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a>
</td>
</table>

## Introdução

Neste colab, vamos abordar como definir as redes personalizadas para seus agentes. As redes nos ajudam a definir o modelo que é treinado pelos agentes. No TF-Agents, você encontrará vários tipos diferentes de redes que são úteis para os agentes:

**Principais redes**

- **QNetwork**: usada no Qlearning para ambientes com ações discretas, essa rede mapeia uma observação para as estimativas de valor de cada ação possível.
- **CriticNetworks**: também conhecida como `ValueNetworks` na literatura, aprende a estimar alguma versão de uma função de valor que mapeia algum estado em uma estimativa para o retorno esperado de uma política. Essas redes estimam o desempenho do estado atual do agente.
- **ActorNetworks**: aprende um mapeamento de observações para ações. Essas redes geralmente são usadas pelas nossas políticas para gerar ações.
- **ActorDistributionNetworks**: parecidas com `ActorNetworks`, mas elas geram uma distribuição que a política pode usar como amostra para gerar ações.

**Redes helper**

- **EncodingNetwork**: permite que os usuários definam um mapeamento de camadas de pré-processamento com facilidade para aplicar a uma entrada de rede.
- **DynamicUnrollLayer**: redefine automaticamente o estado da rede nos limites do episódio, já que é aplicada em uma sequência de tempo.
- **ProjectionNetwork**: redes como `CategoricalProjectionNetwork` ou `NormalProjectionNetwork` aceitam entradas e geram os parâmetros necessários para gerar distribuições categóricas ou normais.

Todos os exemplos no TF-Agents têm redes pré-configuradas. No entanto, essas redes não são configuradas para lidar com observações complexas.

Se você tem um ambiente que expõe mais de uma observação/ação e precisa personalizar suas redes, este tutorial é para você!

## Configuração

Se você ainda não instalou o tf-agents, execute:

In [ ]:
!pip install tf-agents

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import abc
import tensorflow as tf
import numpy as np

from tf_agents.environments import random_py_environment
from tf_agents.environments import tf_py_environment
from tf_agents.networks import encoding_network
from tf_agents.networks import network
from tf_agents.networks import utils
from tf_agents.specs import array_spec
from tf_agents.utils import common as common_utils
from tf_agents.utils import nest_utils

## Como definir as redes

### API Network

No TF-Agents, criamos uma subclasse das [Redes](https://github.com/tensorflow/agents/blob/master/tf_agents/networks/network.py) do Keras. Com isso, podemos:

- Simplificar as operações de cópia necessárias ao criar redes alvo.
- Criar variáveis automaticamente ao chamar `network.variables()`.
- Validar entradas com base em input_specs de rede.

##EncodingNetwork Como mencionado acima, a `EncodingNetwork` permite definir um mapeamento das camadas de pré-processamento com facilidade para aplicar a uma entrada de rede e gerar codificação.

A EncodingNetwork é formada pelas seguintes camadas em maior parte opcionais:

- Camadas de pré-processamento
- Combiner de pré-processamento
- Conv2D
- Flatten
- Dense

A característica especial das redes de encoding é que o pré-processamento de entrada é aplicado. O pré-processamento de entrada é possível pelas camadas `preprocessing_layers` e `preprocessing_combiner`.  Cada uma delas pode ser especificada como uma estrutura aninhada. Se o ninho `preprocessing_layers` for mais superficial que `input_tensor_spec`, as camadas terão subninhos. Por exemplo, se:

```
input_tensor_spec = ([TensorSpec(3)] * 2, [TensorSpec(3)] * 5)
preprocessing_layers = (Layer1(), Layer2())
```

o pré-processamento chamará:

```
preprocessed = [preprocessing_layers[0](observations[0]),
                preprocessing_layers[1](observations[1])]
```

No entanto, se

```
preprocessing_layers = ([Layer1() for _ in range(2)],
                        [Layer2() for _ in range(5)])
```

o pré-processamento chamará:

```python
preprocessed = [
  layer(obs) for layer, obs in zip(flatten(preprocessing_layers),
                                    flatten(observations))
]
```


### Redes personalizadas

Para criar suas próprias redes, você só terá que sobrepor os métodos `__init__` e `call`. Vamos criar uma rede personalizada usando o que aprendemos sobre `EncodingNetworks` para criar uma ActorNetwork que aceita observações contendo uma imagem e um vetor.


In [ ]:
class ActorNetwork(network.Network):

  def __init__(self,
               observation_spec,
               action_spec,
               preprocessing_layers=None,
               preprocessing_combiner=None,
               conv_layer_params=None,
               fc_layer_params=(75, 40),
               dropout_layer_params=None,
               activation_fn=tf.keras.activations.relu,
               enable_last_layer_zero_initializer=False,
               name='ActorNetwork'):
    super(ActorNetwork, self).__init__(
        input_tensor_spec=observation_spec, state_spec=(), name=name)

    # For simplicity we will only support a single action float output.
    self._action_spec = action_spec
    flat_action_spec = tf.nest.flatten(action_spec)
    if len(flat_action_spec) > 1:
      raise ValueError('Only a single action is supported by this network')
    self._single_action_spec = flat_action_spec[0]
    if self._single_action_spec.dtype not in [tf.float32, tf.float64]:
      raise ValueError('Only float actions are supported by this network.')

    kernel_initializer = tf.keras.initializers.VarianceScaling(
        scale=1. / 3., mode='fan_in', distribution='uniform')
    self._encoder = encoding_network.EncodingNetwork(
        observation_spec,
        preprocessing_layers=preprocessing_layers,
        preprocessing_combiner=preprocessing_combiner,
        conv_layer_params=conv_layer_params,
        fc_layer_params=fc_layer_params,
        dropout_layer_params=dropout_layer_params,
        activation_fn=activation_fn,
        kernel_initializer=kernel_initializer,
        batch_squash=False)

    initializer = tf.keras.initializers.RandomUniform(
        minval=-0.003, maxval=0.003)

    self._action_projection_layer = tf.keras.layers.Dense(
        flat_action_spec[0].shape.num_elements(),
        activation=tf.keras.activations.tanh,
        kernel_initializer=initializer,
        name='action')

  def call(self, observations, step_type=(), network_state=()):
    outer_rank = nest_utils.get_outer_rank(observations, self.input_tensor_spec)
    # We use batch_squash here in case the observations have a time sequence
    # compoment.
    batch_squash = utils.BatchSquash(outer_rank)
    observations = tf.nest.map_structure(batch_squash.flatten, observations)

    state, network_state = self._encoder(
        observations, step_type=step_type, network_state=network_state)
    actions = self._action_projection_layer(state)
    actions = common_utils.scale_to_spec(actions, self._single_action_spec)
    actions = batch_squash.unflatten(actions)
    return tf.nest.pack_sequence_as(self._action_spec, [actions]), network_state

Vamos criar um `RandomPyEnvironment` para gerar observações estruturadas e validar nossa implementação.

In [ ]:
action_spec = array_spec.BoundedArraySpec((3,), np.float32, minimum=0, maximum=10)
observation_spec =  {
    'image': array_spec.BoundedArraySpec((16, 16, 3), np.float32, minimum=0,
                                        maximum=255),
    'vector': array_spec.BoundedArraySpec((5,), np.float32, minimum=-100,
                                          maximum=100)}

random_env = random_py_environment.RandomPyEnvironment(observation_spec, action_spec=action_spec)

# Convert the environment to a TFEnv to generate tensors.
tf_env = tf_py_environment.TFPyEnvironment(random_env)

Como definimos as observações como um dicionário, precisamos criar camadas de pré-processamento para lidar com elas.

In [ ]:
preprocessing_layers = {
    'image': tf.keras.models.Sequential([tf.keras.layers.Conv2D(8, 4),
                                        tf.keras.layers.Flatten()]),
    'vector': tf.keras.layers.Dense(5)
    }
preprocessing_combiner = tf.keras.layers.Concatenate(axis=-1)
actor = ActorNetwork(tf_env.observation_spec(), 
                     tf_env.action_spec(),
                     preprocessing_layers=preprocessing_layers,
                     preprocessing_combiner=preprocessing_combiner)

Agora que temos a rede actor, podemos processar as observações do ambiente.

In [ ]:
time_step = tf_env.reset()
actor(time_step.observation, time_step.step_type)

Essa mesma estratégia pode ser usada para personalizar qualquer uma das principais redes usadas pelos agentes. Você pode definir um pré-processamento e conectá-lo ao resto da rede. Ao definir uma rede personalizada, verifique se as definições da camada de saída correspondem.